In [1]:
import numpy as np 
import numba

In [2]:
from sklearn.datasets import load_breast_cancer
X, y = load_breast_cancer(return_X_y=True)

In [3]:
X.shape

(569, 30)

#  KMeans

In [39]:
from math import sqrt

def kmeans(A, numCenter, numIter, N, D, init_centroids):
    centroids = init_centroids

    for l in range(numIter):
        dist = np.array([[sqrt(np.sum((A[i,:]-centroids[j,:])**2))
                                for j in range(numCenter)] for i in range(N)])
        
        labels = np.array([dist[i,:].argmin() for i in range(N)])

        centroids = np.array([[np.sum(A[labels==i, j])/np.sum(labels==i)
                                 for j in range(D)] for i in range(numCenter)])

    return centroids

In [44]:
init_centroids = X[np.random.choice(X.shape[0], size=5), :]

In [46]:
%%timeit

res = kmeans(X, 5, 20, X.shape[0], X.shape[1], init_centroids)

1 loop, best of 3: 431 ms per loop


In [48]:
@numba.njit
def kmeans_numba(A, numCenter, numIter, N, D, init_centroids):
    centroids = init_centroids

    for l in range(numIter):
        dist = np.array([[sqrt(np.sum((A[i,:] - centroids[j,:])**2))
                                for j in range(numCenter)] for i in range(N)])
        
        labels = np.array([dist[i,:].argmin() for i in range(N)])

        centroids = np.array([[np.sum(A[labels==i, j])/np.sum(labels==i)
                                 for j in range(D)] for i in range(numCenter)])

    return centroids

In [50]:
%%timeit

res = kmeans_numba(X, 5, 20, X.shape[0], X.shape[1], init_centroids)

10 loops, best of 3: 19.6 ms per loop


# Logistic Regression 

In [ ]:
def sigmoid(X, weights):
    score = X.dot(weights)
    proba = 1 / (1 + np.exp(-score))

    return proba

def logistic_regression(X, y, maxiter=1000, step_size=1e-10, 
                        l2_penalty=1, regularize=True, add_intercept=False):
    if add_intercept:
        intercept = np.ones((X.shape[0], 1))
        X = np.hstack((intercept, X))
        
    num_param = X.shape[1]

    weights = np.zeros((num_param, 1))
    # indicator = (y == 1)

    for itr in range(maxiter):
        pred_proba = sigmoid(X, weights)
        errors = y.reshape(-1, 1) - pred_proba
        gradient = X.T @ errors
        
        if regularize:
            gradient[1:] -= 2 * l2_penalty * weights[1:]


        weights = weights + step_size*gradient
        
    return weights

In [5]:
%%timeit

weights = logistic_regression(X, y)

10 loops, best of 3: 33.1 ms per loop


In [60]:
@numba.jit(nopython=True)
def logistic_regression_numba(X, y, maxiter, stepsize, l2_penalty, initial_weights):
        
    num_param = X.shape[1]

    weights = initial_weights

    for itr in range(maxiter):
        score = np.dot(X, weights)
        pred_proba = 1 / (1 + np.exp(-score))
        errors = y - pred_proba
        
        gradient = np.dot(X.T, errors)
        
        for j in range(len(gradient), 1):
            gradient[j] -= 2 * l2_penalty * weights[j]

        weights = weights + stepsize*gradient
        
    return weights

In [64]:
%%timeit

weights = logistic_regression_numba(X, y.reshape(-1, 1), maxiter=1000,
                                                        stepsize=1e-10,
                                                        l2_penalty=1,
                                   initial_weights=np.zeros((X.shape[1], 1)))

10 loops, best of 3: 18.5 ms per loop


In [86]:
run_parallel = numba.config.NUMBA_NUM_THREADS > 1

@numba.njit(parallel=False)
def logistic_regression_numba_improved(Y, X, w, iterations):
    for i in range(iterations):
        w -= np.dot(((1.0 / (1.0 + np.exp(-Y * np.dot(X,w))) - 1.0) * Y),X)
    return w

In [ ]:
%%timeit

weights = logistic_regression_numba_improved(y, X, np.zeros(X.shape[1]), 1000)

#  Linear reg

- Example from Numba

Lots more at:

https://github.com/numba/numba/tree/master/examples

In [107]:
from sklearn.datasets import load_boston
from timeit import default_timer as timer


X, Y = load_boston(return_X_y=True)

In [113]:
def gradient_descent_numpy(X, Y, theta, alpha, num_iters):
    m = Y.shape[0]

    theta_x = 0.0
    theta_y = 0.0

    for i in range(num_iters):
        predict = theta_x + theta_y * X
        err_x = (predict - Y)
        err_y = (predict - Y) * X
        theta_x = theta_x - alpha * (1.0 / m) * err_x.sum()
        theta_y = theta_y - alpha * (1.0 / m) * err_y.sum()

    theta[0] = theta_x
    theta[1] = theta_y

In [101]:

from numba import autojit, jit, f8, int32, void

@jit(void(f8[:], f8[:], f8[:], f8, int32))
def gradient_descent_numba(X, Y, theta, alpha, num_iters):
    m = Y.shape[0]

    theta_x = 0.0
    theta_y = 0.0

    for i in range(num_iters):
        err_acc_x = 0.0
        err_acc_y = 0.0
        for j in range(X.shape[0]):
            predict = theta_x + theta_y * X[j]
            err_acc_x += predict - Y[j]
            err_acc_y += (predict - Y[j]) * X[j]
        theta_x = theta_x - alpha * (1.0 / m) * err_acc_x
        theta_y = theta_y - alpha * (1.0 / m) * err_acc_y

    theta[0] = theta_x
    theta[1] = theta_y

In [102]:
def run(gradient_descent, X, Y, iterations=10000, alpha=1e-6):
    theta = np.empty(2, dtype=X.dtype)

    ts = timer()
    gradient_descent(X, Y, theta, alpha, iterations)
    te = timer()

    timing = te - ts

    print("x-offset = {}    slope = {}".format(*theta))
    print("time elapsed: {} s".format(timing))

    return theta, timing

In [114]:
X = X[:, 1]
print('NumPy'.center(30, '-'))
theta_python, time_python = run(gradient_descent_numpy, X, Y)

print('Numba'.center(30, '-'))
theta_numba, time_numba  = run(gradient_descent_numba, X, Y)

# make sure all method yields the same result
assert np.allclose(theta_python, theta_numba)

print('Summary'.center(30, '='))
print('Numba speedup %.1fx' % (time_python / time_numba))

------------NumPy-------------
x-offset = 0.17657380977258894    slope = 0.4927027783996578
time elapsed: 0.16458208899985038 s
------------Numba-------------
x-offset = 0.176573809772589    slope = 0.4927027783996578
time elapsed: 0.007010377999904449 s
===========Summary============
Numba speedup 23.5x
